# Napari demo

## Import packages

Click on **[this link](../desktop)** to open a new window where napari will be visible. Only required for mybinder


In [1]:
import os
os.environ['DISPLAY'] = ':1.0'

import warnings
warnings.filterwarnings('ignore')
import napari
import skimage
import matplotlib.pyplot as plt
import numpy as np
import requests
import skimage.feature
import pandas as pd
import trackpy
import pims
import numpy as np
import dask
import dask.array
import warnings
import cupy as cp
import glob
import Libraries
import sys
sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import cellpose_mask,cellpose_model

from cellpose import io as iocp
from cellpose import plot,utils


2021-07-22 17:02:50,088 [INFO] WRITING LOG OUTPUT TO /home/jmamede/.cellpose/run.log


In [2]:
%gui qt5

## Download the data from Zenodo.

This dataset is a time-lapse of a zebrafish embry acquired by confocal microscopy. See [here](https://zenodo.org/record/1211599#.XZSnAOf7TOR) for more details.

In [2]:

__author__ = """João Mamede"""
__email__ = "jmamede@rush.edu"


# filelist = glob.glob('/run/media/jmamede/Joao/CAruby/20201208caruby/*.ome.tiff')
filelist = glob.glob('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/*ome.tiff')
filelist.sort()

# all = []
filelist
all = Libraries.stitch(filelist,9,5)
red = all[1:-2:3]
green = all[:-3:3]
blue = all[2:-1:3]
#can't reshape I don't know why, resclicing was the only way I found



#DETAIL IS HERE!!!!
rgb = dask.array.stack([red,blue,green],axis=0)


0 odd ['/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v01_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v02_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v03_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v04_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v05_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v06_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v07_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_p

/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v33_PRJ.ome.tiff
/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v32_PRJ.ome.tiff
/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v31_PRJ.ome.tiff
/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v30_PRJ.ome.tiff
/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v29_PRJ.ome.tiff
/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v28_PRJ.ome.tiff
4 odd ['/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v37_PRJ.ome.tiff', '/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/M13d_iGFPCAru3_01062021_PBN_VitC_Rutin_NH4Cl_nuc_Vera_pump_v38_PRJ.ome.ti

In [4]:
rgb

,Array,Chunk
Bytes,34.54 GB,2.10 MB
Shape,"(3, 122, 5120, 9216)","(1, 1, 1024, 1024)"
Count,115965 Tasks,16470 Chunks
Type,uint16,numpy.ndarray


## Image import and plotting

We rescale the image in z by a factor of 5 to clearly show all features. We only keep the later time points for the demo.

In [5]:
# ##USE THIS ON MYBINDER (RESIZED IMAGE AND FEWER TIME POINTS
# '''image_import = skimage.external.tifffile.TiffFile('Sample/cxcr4aMO2_290112.lsm') 
# time_min = 100
# time_max = 130
# image_time = np.zeros((time_max-time_min,22,256,256))
# for t in range(time_min, time_max):
#     image_time[t-time_min,:,:,:] = image_import.asarray(key=2*np.arange(t*44,(t+1)*44))[::2,::2,::2]
# '''  

# ##USE THIS ON A REGULAR COMPUTER
# image_time = skimage.io.imread('Sample/cxcr4aMO2_290112.lsm', 
#                                plugin='tifffile')
# image_time = image_time[0,100:150,:,:,:]

## Nuclei detection
We use a simple gaussian filtering followed by local maxima detection to find all the nuclei and create an array with time, x, y,z positions.

In [6]:
# model_nuc = cellpose_model(GPU=True,model_type='nuclei')

** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off


In [7]:
# nuclei_masks = cellpose_mask(blue[0].compute(),model_nuc
# #                             ,size=2500
#                             ,flow_threshold=1.0,size=0,diam=50,cell_prob=-1.2,
#                              _bin=False,
#                            )

processing 1 image(s)
time spent: running network 21.19s; flow+mask computation 18.68
estimated masks for 1 image(s) in 40.97 sec
>>>> TOTAL TIME 40.97 sec


In [5]:
# plt.imshow(nuclei_masks)

In [5]:

import scipy.ndimage as ndi

In [10]:
# coords = ndi.measurements.center_of_mass(blue[0].compute(),nuclei_masks ,np.arange(1,nuclei_masks .max()+1))

In [7]:
# all_dots = []
# labels = np.zeros((blue.shape))
# for i in range(blue.shape[0]):
#     print(i)
#     nuclei_masks = cellpose_mask(blue[i].compute(),model_nuc
# #                             ,size=2500
#                             ,flow_threshold=1.2,size=0,diam=50,cell_prob=-1.2,
#                              _bin=False,
#                            )
#     coords = np.stack(
#                 ndi.measurements.center_of_mass(
#                     blue[i].compute(),nuclei_masks ,np.arange(1,nuclei_masks .max()+1))
#     )
#     labels[i,:,:] = nuclei_masks
#     all_dots.append(np.stack((i*np.ones(coords.shape[0]),
#                        coords[:,0],coords[:,1]),axis =1))
# spots = np.concatenate(all_dots)
# spots = np.asarray(all_dots)

In [9]:
# spots = np.concatenate(all_dots)
# spots_pd = pd.DataFrame(spots,columns=['frame','x','y'])


In [21]:
store.close()

In [3]:
spots = pd.read_hdf('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/Nuclei_spots3.h5')

2021-07-22 17:03:01,167 [INFO] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-07-22 17:03:01,168 [INFO] NumExpr defaulting to 8 threads.


In [4]:
spots

,frame,x,y
0,0.0,2.978210,7809.774904
1,0.0,5.803889,4709.715707
2,0.0,9.232053,4080.731574
3,0.0,9.128707,6866.842092
4,0.0,12.317972,6304.847350
...,...,...,...
230078,121.0,5108.168657,3058.524973
230079,121.0,5108.508426,6699.743452
230080,121.0,5113.261983,3746.941546
230081,121.0,5113.304121,5098.750864


In [7]:
traj = pd.read_hdf('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/TrajectoriesNuclei_65range.h5')

all_traj = []
for ind in traj.particle.unique():
    curr_traj = traj[traj.particle==ind][['y','x']].values
    if curr_traj.shape[0]>5:
        all_traj.append(curr_traj)

In [8]:
all_traj

[array([[3347.28703085, 2048.2801696 ],
        [3353.21372639, 2046.73470348],
        [3353.08041545, 2046.70981642],
        [3349.76301557, 2050.27555836],
        [3348.19020492, 2053.56646978],
        [3345.23053907, 2060.96441274],
        [3340.66618315, 2067.54570422],
        [3339.18458483, 2067.97404167],
        [3341.12395778, 2066.57586859],
        [3338.55372113, 2070.86201701],
        [3329.09218095, 2068.0634988 ],
        [3319.77900324, 2085.65415198],
        [3321.98681073, 2093.07313206],
        [3311.83666497, 2095.30055454],
        [3322.25539529, 2096.71101892],
        [3342.80638571, 2077.56741104],
        [3350.93988193, 2062.63777784],
        [3350.54794304, 2058.33212294],
        [3359.12577826, 2063.78771265],
        [3353.1143315 , 2067.67775087],
        [3352.87786611, 2068.52498966],
        [3350.91612295, 2077.01810703],
        [3344.72625223, 2087.65899063],
        [3345.84924024, 2087.10589611],
        [3351.1786547 , 2083.34677287],


In [12]:
# import tifffile as tf
# tf.imwrite('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/labels.tiff', labels,
#                        compress='LZMA',
#                        photometric='minisblack',
#                         #datetime= True,
#                         metadata={'axes': 'TYX'},
#                         contiguous=False)
green = green.compute()
red = red.compute()
blue = blue.compute()

In [9]:
v = napari.Viewer(ndisplay=2)
rmin = 1000
rmax = 15000
gmin = 1000
gmax = 15000
bmin = 1000
bmax = 30000

v.add_image(green,
            # rgb=True,
#             contrast_limits=[rmin,rmax],
            blending='additive',
            colormap='green',
            name='HIV-iGFP',#, is_pyramid=False
                 )
v.add_image(red,
#             contrast_limits=[gmin,gmax],
        blending='additive',
        colormap='red',
        name='CA-mRuby3',#, is_pyramid=False
             )
v.add_image(blue, 
#             contrast_limits=[bmin,bmax],
        blending='additive',
        colormap='blue',
        name='Nucspot650',#, is_pyramid=False
             )

v.add_points(spots,face_color='red',opacity=0.5,size=15)
v.add_shapes(all_traj, shape_type='path',edge_color='grey')

DeflateError: libdeflate_zlib_decompress returned LIBDEFLATE_INSUFFICIENT_SPACE

In [11]:
import time
metadata = pims.Bioformats(filelist[0]).metadata
metadata.PlaneDeltaT(0,0)
timelist = []
for i in range(metadata.PlaneCount(0)):
    timelist.append(metadata.PlaneDeltaT(0,i))

timelist = np.asarray(timelist)
timelist = timelist.reshape((metadata.PlaneCount(0)//3,metadata.ChannelCount(0)))
timelist.shape
timelist = timelist[:,0]
del(metadata)

import time
def update_slider(event):
    import time
    # only trigger if update comes from first axis (optional)
#     print('inside')
        #ind_lambda = viewer.dims.indices[0]
    ind_lambda = v.dims.current_step[0]
    timestrg = "Day: {0}".format(time.strftime('%d %H:%M:%S',time.gmtime(timelist[ind_lambda])))
    v.text_overlay.text = timestrg


In [12]:
f_igfp = pd.read_hdf('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/spots_igfp_good.h5')
f_caruby = pd.read_hdf('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/spots_caru3_good.h5')
f_caruby ['x'] = f_caruby ['x']*0.108
f_caruby ['y'] = f_caruby ['y']*0.108
f_igfp['x'] = f_igfp['x']*0.108
f_igfp['y'] = f_igfp['y']*0.108
spots['x'] = spots['x']*0.108
spots['y'] = spots['y']*0.108

v.text_overlay.visible = True
v.text_overlay.text = "Day:{0}".format(time.strftime('%d %H:%M:%S',time.gmtime(timelist[0])))

v.dims.events.current_step.connect(update_slider)

# v.add_points(spots,face_color='white',opacity=0.5,size=1.5)

v.add_points(spots_igfp,face_color='green',opacity=0.5,size=0.4)
v.add_points(spots_caruby,face_color='red',opacity=0.5,size=0.4)

NameError: name 'spots_igfp' is not defined

In [48]:
v.add_image(labels,
        blending='additive',
        colormap='jet',
        name='masks',#, is_pyramid=False
             )

# v.add_image(distance,
#         blending='additive',
#         colormap='jet',
#         name='distance',#, is_pyramid=False
#              )

# v.add_image(local_maxi,
#         blending='additive',
#         colormap='red',
#         name='maxi',#, is_pyramid=False
#              )


# v.add_image(labels,
#         blending='additive',
#         colormap='jet',
#         name='localmax',#, is_pyramid=False
#              )

# v.add_points(coords,face_color='white',opacity=0.5,size=15)

<Image layer 'masks' at 0x7f7833fd6910>

In [11]:
traj = pd.read_hdf('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/TrajectoriesNuclei_65range.h5')

all_traj = []
for ind in traj.particle.unique():
    curr_traj = traj[traj.particle==ind][['y','x']].values
    if curr_traj.shape[0]>5:
        all_traj.append(curr_traj)

## Tracking

Finally we use [trackpy](https://soft-matter.github.io/trackpy/v0.3.2/) to track the different nuclei. For that we just have to turn our time,x,y,z array into a Pandas dataframe and then call trackpy:

In [28]:
# spots_pd = pd.DataFrame(spots,columns=['frame','y','x'])
traj = trackpy.link_df(spots_pd,search_range=65,memory=2)
# with trackpy.PandasHDFStore('/run/media/jmamede/Joao/CAruby/20210406mdm-WOWO/TrajectoriesNuclei_65range.h5') as s:
#     s.put(traj)

Frame 121: 2034 trajectories present.


We just exclude very short tracks to make the plot more readable:

And finally we add the trajectories to the plot as lines:

In [396]:
del(blue)
all8 = all.map_blocks(Libraries.convert16to8bits)
green = all8[:-3:3]
red = all8[1:-2:3]
blue = all8[2:-1:3]
green.shape

blue = blue.compute()
red = red.compute()
green = green.compute()

In [399]:
blue = rgb[2,...]
red = rgb[1,...]
green = rgb[0,...]

In [401]:
# rmin = np.percentile(red,0.5)
# rmax = np.percentile(red,99.9)
# gmin = np.percentile(green,0.5)
# gmax = np.percentile(green,99.9)
# bmin = np.percentile(blue,0.5)
# bmax = np.percentile(blue,99.9)
rmin = 1000
rmax = 15000
gmin = 1000
gmax = 15000
bmin = 1000
bmax = 30000

# rmin = 0
# rmax = 255
# gmin = 0
# gmax = 255
# bmin = 0
# bmax = 255
v = napari.Viewer(ndisplay=2)
v.add_image(green,
            # rgb=True,
            contrast_limits=[rmin,rmax],
            blending='additive',
            colormap='green',
            name='HIV-iGFP',#, is_pyramid=False
                 )
v.add_image(red, contrast_limits=[gmin,gmax],
        blending='additive',
        colormap='red',
        name='CA-mRuby3',#, is_pyramid=False
             )
v.add_image(blue, contrast_limits=[bmin,bmax],
        blending='additive',
        colormap='blue',
        name='Nucspot650',#, is_pyramid=False
             )

v.add_points(spots,face_color='red',opacity=0.5,size=10)
v.add_shapes(all_traj, shape_type='path',edge_color='grey')

<Shapes layer 'all_traj' at 0x7f6d49965c10>